# 🚀 Stable Diffusion WebUI com LoRAs no Colab (modo CPU)

In [ ]:
# 📂 Montar Google Drivefrom google.colab import drivedrive.mount('/content/drive')

In [ ]:
# 🔧 Dependências principais (forçando reinstalação limpa)!pip install --upgrade pip!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu!pip install pytorch-lightning==2.4.0 --no-cache-dir --force-reinstall

In [ ]:
# ✅ Verificar instalaçãoimport pytorch_lightning as plprint('PyTorch Lightning carregado:', pl.__version__)

In [ ]:
# ⬇️ Clonar Automatic1111!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

In [ ]:
# 🔗 Vincular pasta de LoRAs do Drive!mkdir -p /content/stable-diffusion-webui/models/Lora!ln -s /content/drive/MyDrive/SD_LoRAs /content/stable-diffusion-webui/models/Lora

In [ ]:
# ▶️ Rodar WebUI em modo CPU%cd /content/stable-diffusion-webui!COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --precision full --no-half" \  REQS_FILE="requirements.txt" \  python launch.py

In [ ]:
# ⏳ Anti-Idle seguro (mantém Colab ativo)import time, threadingdef anti_idle():    while True:        time.sleep(60)threading.Thread(target=anti_idle, daemon=True).start()print('✅ Anti-Idle ativado!')